In [142]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import affinity
from shapely.geometry import * 

# functions

In [143]:
def get_angle(a, b, c):
    # based on https://stackoverflow.com/questions/61439009/rotate-alignment-an-image-by-line-along-the-y-axis
    
    if LineString([a, b]).length > LineString([b, c]).length:
        lon_len = LineString([a,b])
    else: 
        lon_len = LineString([b,c])
    
    P1 = np.array(lon_len.coords[0])
    P2 = np.array(lon_len.coords[1])

    if (P2[1] - P1[1]) < 0:
        if (P2[0] - P1[0]) < 0:
            y_axis_vector = np.array([0, -1])
            #print('1')
        else:
            y_axis_vector = np.array([0, 1])
            #print('2')
    else:
        if (P2[0] - P1[0]) < 0:
            y_axis_vector = np.array([0, -1])
            #print('3')
        else:
            y_axis_vector = np.array([0, 1])
            #print('4')

    p_unit_vector = (P2 - P1) / np.linalg.norm(P2-P1)

    return np.arccos(np.dot(p_unit_vector, y_axis_vector)) * 180 /math.pi

In [4]:
def rotate_to_y_axis(geom):
    mbr = geom.minimum_rotated_rectangle
    a, b, c = mbr.exterior.coords[0:3]
    rotation_angle = get_angle(a, b, c)
    return rotation_angle

In [5]:
def get_long(mrr):
    a, b, c = mrr.exterior.coords[0:3]
    
    if LineString([a, b]).length > LineString([b, c]).length:
        return LineString([a,b]).length
    else: 
        return LineString([b,c]).length

In [6]:
def get_short(mrr):
    a, b, c = mrr.exterior.coords[0:3]
    
    if LineString([a, b]).length < LineString([b, c]).length:
        return LineString([a,b]).length
    else: 
        return LineString([b,c]).length

In [7]:
def import_reference_dict(path):
    char_models_df = gpd.read_file(path)
    
    char_models_df.set_index('name')

    char_models_df['area'] = char_models_df['geometry'].area
    char_models_df['mrr'] = gpd.GeoSeries([b.minimum_rotated_rectangle for b in char_models_df['geometry']])
    char_models_df['mrr_area'] = char_models_df['mrr'].area
    char_models_df['centroid'] = char_models_df['mrr'].centroid
    char_models_df['mrr_long'] = pd.Series([get_long(mrr) for mrr in char_models_df['mrr']])
    char_models_df['mrr_short'] = pd.Series([get_short(mrr) for mrr in char_models_df['mrr']])

    char_models_df['rotate_angle'] = pd.Series([rotate_to_y_axis(geo) for geo in char_models_df['geometry']])
    char_models_df['rotate_angle'] = pd.Series([angle if name != 'Z' else angle + 30 for name, angle in char_models_df[['name', 'rotate_angle']].values])
    char_models_df['rotated_geom'] = gpd.GeoSeries([affinity.rotate(geo, angle) for geo, angle in char_models_df[['geometry', 'rotate_angle']].values])
    
    char_models_df['geom_translate'] =  gpd.GeoSeries([affinity.translate(geo, -centroid.x, -centroid.y) for centroid, geo in char_models_df[['centroid', 'rotated_geom']].values])
    char_models_df['geom_norm'] =  gpd.GeoSeries([affinity.scale(geo, xfact = 1/xf, yfact = 1/yf, origin=(0,0)) for xf, yf, geo in char_models_df[['mrr_short', 'mrr_long', 'geom_translate']].values])
    char_models_df['norm_centroid'] = char_models_df['geom_norm'].centroid
    
    char_models = {char_models_df['name'].tolist()[i]: char_models_df['geom_norm'].tolist()[i] for i in range(len(char_models_df['name'].tolist()))}
    reference_chars = {}
    for c in ('O'):
        reference_chars[c] = [char_models[c]]
    for c in ('I', 'H'):
        reference_chars[c] = [char_models[c]]
        reference_chars[c].append(affinity.rotate(char_models[c], 90))
    for c in ('E', 'T', 'U', 'Y'):
        reference_chars[c] = [char_models[c]]
        for i in (90, 180, 270):
            reference_chars[c].append(affinity.rotate(char_models[c], i))
    for c in ('F', 'L', 'Z'):
        reference_chars[c] = [char_models[c]]
        #for i in (45, 90, 135, 180, 225, 270, 315):
        for i in (90, 180, 270):
            reference_chars[c].append(affinity.rotate(char_models[c], i))
        geom_mirror = affinity.scale(char_models[c], xfact = -1.0, yfact = 1.0)
        reference_chars[c].append(geom_mirror)
        #for i in (45, 90, 135, 180, 225, 270, 315):
        for i in (90, 180, 270):
            reference_chars[c].append(affinity.rotate(geom_mirror, i))
    return reference_chars

In [8]:
def preprocess_polygons(polygon_df):
    polygon_df['area'] = polygon_df['geometry'].area
    polygon_df['mrr'] = gpd.GeoSeries([b.minimum_rotated_rectangle for b in polygon_df['geometry']])
    polygon_df['mrr_area'] = polygon_df['mrr'].area
    polygon_df['centroid'] = polygon_df['mrr'].centroid
    polygon_df['mrr_long'] = pd.Series([get_long(mrr) for mrr in polygon_df['mrr']])
    polygon_df['mrr_short'] = pd.Series([get_short(mrr) for mrr in polygon_df['mrr']])

    polygon_df['rotate_angle'] = pd.Series([rotate_to_y_axis(geo) for geo in polygon_df['geometry']])
    polygon_df['rotated_geom'] = gpd.GeoSeries([affinity.rotate(geo, angle) for geo, angle in polygon_df[['geometry', 'rotate_angle']].values])
    polygon_df['geom_translate'] =  gpd.GeoSeries([affinity.translate(geo, -centroid.x, -centroid.y) for centroid, geo in polygon_df[['centroid', 'rotated_geom']].values])
    polygon_df['scale'] = pd.Series([1/math.sqrt(area) for area in polygon_df['mrr_area']])
    polygon_df['geom_norm'] =  gpd.GeoSeries([affinity.scale(geo, xfact = 1/xf, yfact = 1/yf, origin=(0,0)) for xf, yf, geo in polygon_df[['mrr_short', 'mrr_long', 'geom_translate']].values])
    return polygon_df

In [128]:
def match_df(geom, mrr_s, mrr_l, angle, char, reference_dict):
    best_match_geom = None
    best_match_iou = 0
    
    geom = geom
    xf = mrr_s
    yf = mrr_l
    angle = angle
    
    for c in reference_dict[char]:
        c_s = affinity.scale(c, xfact = xf, yfact = yf, origin=c.minimum_rotated_rectangle.centroid)
        c_sr = affinity.rotate(c_s, 180-angle)
        g_sr = c_sr.minimum_rotated_rectangle.centroid
        c_srt = affinity.translate(c_sr, geom.minimum_rotated_rectangle.centroid.x-g_sr.x, geom.minimum_rotated_rectangle.centroid.y-g_sr.y)
        iou = geom.intersection(c_srt).area/geom.union(c_srt).area
        if iou > best_match_iou:
            best_match_geom = c_srt
            best_match_iou = iou
        
        if char in ('F', 'L', 'Z'):
            for i in (45, 90, 135, 180, 225, 270, 315):
            #for i in range(0,360,1):    
                c_srt_ = affinity.rotate(c_srt, i)
                iou_ = geom.intersection(c_srt_).area/geom.union(c_srt_).area
                if iou_ > best_match_iou:
                    best_match_geom = c_srt_
                    best_match_iou = iou_
        
    #print(best_match_iou)
    return best_match_geom

# case study

In [140]:
reference_chars = import_reference_dict(path = "data/10_templates.shp")
path_to_geometries = 'data/case_study.shp'
path_to_predictions = 'path to file with class predictions'
path_to_results = 'path to file to save results'

In [141]:
for model in ('CNN', 'RNN', 'GCNN'):
    buildings_beijing = gpd.read_file(path_to_geometries)
    beijing = pd.read_csv(path_to_predictions+model+".csv")
    buildings_beijing = preprocess_polygons(buildings_beijing)
    buildings_beijing['osm_id']=buildings_beijing['osm_id'].astype('int')
    #buildings_beijing['osm_id']
    beijing['osm_id_ret'] = beijing['osm_id_ret'].astype('int')
    df = pd.merge(buildings_beijing, beijing, left_on='osm_id', right_on = 'osm_id_ret')
    df['match_geom'] = gpd.GeoSeries([match_df(geom, mrr_s, mrr_l, angle, char, reference_chars) 
                                      for geom, mrr_s, mrr_l, angle, char 
                                      in df[['geometry', 'mrr_short', 'mrr_long', 'rotate_angle', 'shape_prediction']].values])
    df_towrite = df.set_geometry('match_geom', crs = 'epsg:25832')
    df_towrite = df_towrite[['osm_id', 'match_geom']]
    df_towrite.to_csv(path_to_results+model+'.csv')